In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


In [ ]:
test=pd.read_csv('/content/drive/MyDrive/quant22/test.csv')
train=pd.read_csv('/content/drive/MyDrive/quant22/train.csv')
dummy = train.copy()
test2 = test.copy()

In [ ]:
test.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [ ]:
dummy.drop('ID_code', axis=1, inplace=True)
correlations = dummy.corr()
print(correlations)

           target     var_0     var_1     var_2     var_3     var_4     var_5  \
target   1.000000  0.052390  0.050343  0.055870  0.011055  0.010915  0.030979   
var_0    0.052390  1.000000 -0.000544  0.006573  0.003801  0.001326  0.003046   
var_1    0.050343 -0.000544  1.000000  0.003980  0.000010  0.000303 -0.000902   
var_2    0.055870  0.006573  0.003980  1.000000  0.001001  0.000723  0.001569   
var_3    0.011055  0.003801  0.000010  0.001001  1.000000 -0.000322  0.003253   
...           ...       ...       ...       ...       ...       ...       ...   
var_195  0.028285  0.002073 -0.000785 -0.001070  0.001206  0.003706 -0.001274   
var_196  0.023608  0.004386 -0.000377  0.003952 -0.002800  0.000513  0.002880   
var_197 -0.035303 -0.000753 -0.004157  0.001078  0.001164 -0.000046 -0.000535   
var_198 -0.053000 -0.005776 -0.004861 -0.000877 -0.001651 -0.001821 -0.000953   
var_199  0.025434  0.003850  0.002287  0.003855  0.000506 -0.000786  0.002767   

            var_6     var_7

In [ ]:
corr_matrix=correlations.abs()
max_corr = (corr_matrix.abs().stack().reset_index()
            .rename(columns={0: 'correlation', 'level_0': 'column1', 'level_1': 'column2'})
            .loc[lambda x: x['column1'] != x['column2']]
            .loc[lambda x: x['correlation'].idxmax()])

print("Pair of columns with maximum correlation:")
print(max_corr)

Pair of columns with maximum correlation:
column1          target
column2          var_81
correlation    0.080917
Name: 82, dtype: object


In [ ]:
train_is_null = dummy.isnull().sum()
print(train_is_null)

target     0
var_0      0
var_1      0
var_2      0
var_3      0
          ..
var_195    0
var_196    0
var_197    0
var_198    0
var_199    0
Length: 201, dtype: int64


In [ ]:
null_count=0
for i in train_is_null:
  if i:
    null_count+=1
print(null_count)

0


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [ ]:
features = dummy.columns.drop('target')
x = dummy.loc[:, features].values
x = StandardScaler().fit_transform(x)

In [ ]:
print(features)

Index(['var_0', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7',
       'var_8', 'var_9',
       ...
       'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195',
       'var_196', 'var_197', 'var_198', 'var_199'],
      dtype='object', length=200)


In [ ]:
def replace_outliers_with_percentiles(data, lower_percentile=10, upper_percentile=90):
    for column in data.columns:
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        lower_percentile_value = data[column].quantile(lower_percentile / 100)
        upper_percentile_value = data[column].quantile(upper_percentile / 100)
        data.loc[data[column] < lower_bound, column] = lower_percentile_value
        data.loc[data[column] > upper_bound, column] = upper_percentile_value
    return data
dummy= replace_outliers_with_percentiles(dummy)

In [ ]:
pca = PCA(n_components=30 )
principal_components = pca.fit_transform(x)
train1 = pd.DataFrame(data = principal_components)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, roc_curve, roc_auc_score

In [ ]:
dummy.head()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [ ]:
train1['target'] = dummy['target']
print(train1.columns)

Index([       0,        1,        2,        3,        4,        5,        6,
              7,        8,        9,       10,       11,       12,       13,
             14,       15,       16,       17,       18,       19,       20,
             21,       22,       23,       24,       25,       26,       27,
             28,       29, 'target'],
      dtype='object')


In [ ]:
X = train1.drop(columns='target')
y = train1['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestClassifier(n_estimators=15, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
y_pred_prob_cv = cross_val_predict(model, X, y, cv=cv, method='predict_proba')[:, 1]
cv_scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
print(f'Cross-validated accuracy scores: {cv_scores}')
print(f'Mean cross-validated accuracy: {cv_scores.mean()}')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Cross-validated accuracy scores: [0.90215  0.902025 0.901475 0.902    0.9014  ]
Mean cross-validated accuracy: 0.90181
Accuracy: 0.898325
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.99      0.95     35903
           1       0.51      0.14      0.22      4097

    accuracy                           0.90     40000
   macro avg       0.71      0.56      0.58     40000
weighted avg       0.87      0.90      0.87     40000



implementing MI and RF importance:

In [ ]:
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
mi_scores = mutual_info_regression(X, y)
mi_scores_series = pd.Series(mi_scores, index=X.columns)
mi_scores_series = mi_scores_series.sort_values(ascending=False)
top_features = mi_scores_series.head(15).index.tolist()
top_features = [feature for feature in top_features if feature in dummy.columns]

mi_dummy = dummy[top_features]


In [ ]:
importances = model.feature_importances_
importances_series = pd.Series(importances, index=X.columns)
indices = np.argsort(importances)[::-1]
top_featuresss = importances_series.head(15).index.tolist()
top_features = [feature for feature in top_featuresss if feature in dummy.columns]

rfi_dummy= dummy[top_features]

# Doing it all on test

In [ ]:
test.drop('ID_code', axis=1, inplace=True)

In [ ]:
test= replace_outliers_with_percentiles(test)

In [ ]:
train_is_null = test.isnull().sum()
print(train_is_null)

var_0      0
var_1      0
var_2      0
var_3      0
var_4      0
          ..
var_195    0
var_196    0
var_197    0
var_198    0
var_199    0
Length: 200, dtype: int64


In [ ]:
features = test.columns
x = dummy.loc[:, features].values
x = StandardScaler().fit_transform(x)

In [ ]:
pca = PCA(n_components=30 )
principal_components = pca.fit_transform(x)
test1 = pd.DataFrame(data = principal_components)

In [ ]:
test1.head()


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,-0.880562,0.023163,0.792230,-1.543798,0.476941,-1.606313,-2.075585,-0.096015,0.265281,0.965948,...,0.165055,-0.111631,0.504462,0.386248,0.853077,-1.406760,-1.979328,0.044179,0.618790,-0.499775
1,1.101024,0.119211,-1.800978,0.345724,-0.646275,-0.079232,-0.977259,-0.307248,-0.980943,0.947005,...,0.221543,-0.392058,-0.273615,-0.496697,-1.398504,0.610482,0.249088,0.867241,0.423077,-0.207489
2,-0.677368,1.559696,0.295904,0.700310,0.586156,-0.900114,0.192544,0.303667,-0.613214,2.144148,...,-1.379762,-0.113868,0.265926,0.955253,0.007608,0.726478,0.144892,-1.561116,0.975062,-0.320431
3,0.644429,-0.137121,-0.457659,1.393919,-0.218645,-0.062603,-0.171860,1.417953,0.535818,-0.947137,...,-0.592216,-0.257628,0.028366,0.281520,-1.032052,0.773462,-0.919598,1.973856,-0.958023,-0.874388
4,1.670343,0.606304,0.180396,0.794673,2.080758,0.460747,0.514343,-1.036062,-0.768949,0.191834,...,0.378579,0.671439,0.426640,-0.898729,0.574220,-0.159475,0.169335,-0.097261,0.597941,0.796362


In [ ]:
print(len(test1))

200000


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, roc_curve, roc_auc_score

In [ ]:
test1.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,-0.880562,0.023163,0.792230,-1.543798,0.476941,-1.606313,-2.075585,-0.096015,0.265281,0.965948,...,0.165055,-0.111631,0.504462,0.386248,0.853077,-1.406760,-1.979328,0.044179,0.618790,-0.499775
1,1.101024,0.119211,-1.800978,0.345724,-0.646275,-0.079232,-0.977259,-0.307248,-0.980943,0.947005,...,0.221543,-0.392058,-0.273615,-0.496697,-1.398504,0.610482,0.249088,0.867241,0.423077,-0.207489
2,-0.677368,1.559696,0.295904,0.700310,0.586156,-0.900114,0.192544,0.303667,-0.613214,2.144148,...,-1.379762,-0.113868,0.265926,0.955253,0.007608,0.726478,0.144892,-1.561116,0.975062,-0.320431
3,0.644429,-0.137121,-0.457659,1.393919,-0.218645,-0.062603,-0.171860,1.417953,0.535818,-0.947137,...,-0.592216,-0.257628,0.028366,0.281520,-1.032052,0.773462,-0.919598,1.973856,-0.958023,-0.874388
4,1.670343,0.606304,0.180396,0.794673,2.080758,0.460747,0.514343,-1.036062,-0.768949,0.191834,...,0.378579,0.671439,0.426640,-0.898729,0.574220,-0.159475,0.169335,-0.097261,0.597941,0.796362


In [ ]:
test2.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [ ]:
test1['ID_code']= test2['ID_code']

In [ ]:
from google.colab import files
y_test1_pred = model.predict(test1.drop(columns=['ID_code']))
submit = pd.DataFrame({
    'ID_code': test1['ID_code'],
    'target': y_test1_pred
})
submit.to_csv('submit.csv', index=False)
files.download('submit.csv')

print('Submission file created')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submission file created
